In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import TweetTokenizer
from wordcloud import WordCloud
from string import punctuation
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
tweets = pd.read_csv('/kaggle/input/pfizer-vaccine-tweets/vaccination_tweets.csv')

In [ ]:
tweets.shape

In [ ]:
tweets.info()

In [ ]:
# Text Cleaning
stop_nltk= stopwords.words("english")
stop_updated= stop_nltk+["https",'t.co','...']
lemm= WordNetLemmatizer()
tweet_tok=  TweetTokenizer()

def clean_text(text):
    tokens= tweet_tok.tokenize(text.lower())
    tokens1 = [re.sub('^https://t.co/[\w]+','',tok) for tok in tokens]
    tokens2 = [re.sub('^@[\w]+','',tok) for tok in tokens1]
    tokens3 = [re.sub('^#[\w]+','',tok) for tok in tokens2]
    tokens4 = [re.sub('[0-9]+','',tok) for tok in tokens3]
    lemmed=[lemm.lemmatize(term) for term in tokens4 if term not in  stop_updated and term not in list(punctuation) and len(term)>2]
    res=' '.join(lemmed)
    return res

In [ ]:
tweets['Clean_Tweets']= tweets['text'].apply(clean_text)

## Top 100 most commonly used words in Tweets

In [ ]:
clean_tweets= "".join(tweets['Clean_Tweets'].values)
word_cloud = WordCloud(width=1200,height=700,
                       background_color='beige',colormap='inferno',
                       max_words=100).generate(clean_tweets)

plt.figure(figsize=[10,15])
plt.title("Top 100 most used words")
plt.imshow(word_cloud)
plt.show()

## Top 20 Hashtags

In [ ]:
tweets['hashtags'] = tweets['hashtags'].astype('str')
tweets['clean_hashtags'] = tweets['hashtags'].apply(lambda x: x[1:-2])
tokens = [re.sub("'","",token) for token in tweets['clean_hashtags'].values]
clean_hashtag = "".join(tokens)
word_cloud = WordCloud(width=500,height=250,
                       background_color='ivory',colormap='inferno',
                       max_words=20).generate(clean_hashtag)

plt.figure(figsize=[10,15])
plt.imshow(word_cloud)
plt.title("Top 20 Hashtags")
plt.show()

## Number of Verified Users in the Tweet pool

In [ ]:
a = pd.DataFrame(tweets['user_verified'].value_counts())
plt.figure(figsize=(7,7))
plt.pie(a['user_verified'],
        labels=['Non-Verified','Verified'],
        autopct='%1.1f%%',
        colors=['steelblue','lightskyblue'])
plt.title("Percentage of Verified Users")

## Conclusion

* 'Vaccine', 'covid', 'vaccination', 'people' & 'first' are the most commonly used words in the Tweets
* Top hashtags are 'PfizerBioNTech', 'COVID19' & 'CovidVaccine'
* It is observed that 9.9% of the Tweets are from Verified Users of Twitter